In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append("../")
from experiments.utils.cd_information import distance, clean_db
from modules.Deduplicator import Deduplicator

CD_DATASET_PATH = '../datasets/cd_information'

# Experiment

In [2]:
df = pd.read_csv(f'{CD_DATASET_PATH}/cd.csv', delimiter=';', doublequote=False)

# evalutation
gold_standard_df = pd.read_csv(f'{CD_DATASET_PATH}/cd_gold.csv', delimiter=';')
gold_standard_pairs = gold_standard_df.values.tolist()
gold_standard_pairs = [tuple(row) for row in gold_standard_pairs]

# DATASET 
df = clean_db(df)

print('Base toda', len(df))

Base toda 9763


In [3]:
# partitions
total_rows = len(df)
first_partition_rows = int(total_rows * 0.50) # 50%
second_partition_rows = int(total_rows * 0.35) # 35%
third_partition_rows = total_rows - first_partition_rows - second_partition_rows # 15%

# Generate random indices for shuffling
np.random.seed(42)
indices = np.random.permutation(total_rows)

# Split the indices into three partitions
first_partition_indices = indices[:first_partition_rows]
second_partition_indices = indices[first_partition_rows:first_partition_rows+second_partition_rows]
third_partition_indices = indices[first_partition_rows+second_partition_rows:]

# Create the three partitions
df1 = df.loc[first_partition_indices]
df2 = df.loc[second_partition_indices]
df3 = df.loc[third_partition_indices]


print('Base particao 1', len(df1))
print('Base particao 2', len(df2))
print('Base particao 3', len(df3))

Base particao 1 4881
Base particao 2 3417
Base particao 3 1465


### BATCH

In [4]:
deduplicator = Deduplicator('title', distance, 'pk', 0.25)

clusters = deduplicator.run(df)

In [5]:
deduplicator.evaluate(gold_standard_pairs)

~~ EVALUATION ~~
  Precision: 0.963855421686747
  Recall: 0.802675585284281
  F-measure: 0.8759124087591241

  TP: 240
  FP: 9
  TN: -1111
  FN: 59

Comparações:  44246


### Incremental

In [6]:
deduplicator_inc = Deduplicator('title', distance, 'pk', 0.25)

In [7]:
clusters_inc = deduplicator_inc.run(df1)

In [8]:
clusters_inc = deduplicator_inc.run(df2)

In [9]:
clusters_inc = deduplicator_inc.run(df3)

In [10]:
deduplicator_inc.evaluate(gold_standard_pairs)

~~ EVALUATION ~~
  Precision: 0.963855421686747
  Recall: 0.802675585284281
  F-measure: 0.8759124087591241

  TP: 240
  FP: 9
  TN: -1111
  FN: 59

Comparações:  45096
